In [ ]:
! pip install -q transformers==4.52.2
! pip install -q -U datasets
! pip install -q peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you ha

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig
from transformers import TrainingArguments, Trainer, TrainerCallback

import sys
sys.path.append('/content/drive/MyDrive/Transformers/Project_OPT')

from OPT_to_GQA import convert_opt_to_gqa

# **Load Model Converted To GQA**
Using the adapted loader for been able to load the model with his GQAAttention conversion.
Also this will give us the ability to load the model as QLORA.


In [ ]:
def create_standard_heads_grouping_arr(num_layers: int, num_heads: int, kv_heads: int):
    assert num_heads % kv_heads == 0, "num_heads must be divisible by kv_heads"
    group_size = num_heads // kv_heads
    grouping = [i for i in range(kv_heads) for _ in range(group_size)]
    return [grouping.copy() for _ in range(num_layers)]

kv_heads = 16
num_heads = 32
group_size = num_heads // kv_heads
num_layers = 32 # 1.3b - 24, 6.7b - 32
heads_grouping_arr = create_standard_heads_grouping_arr(num_layers, num_heads,
                                                        kv_heads)

repo_size = "6.7b"
saved_path = f"facebook-opt-{repo_size}-GQA-{kv_heads}-kv"

In [ ]:
from torch import nn
from transformers.models.opt.modeling_opt import OPTForSequenceClassification, OPTModel, OPTDecoder, OPTDecoderLayer
from OPT_to_GQA import OPTGQAAttention

class CustomOPTDecoderLayer(OPTDecoderLayer):
    def __init__(self, config, layer_idx):
        super().__init__(config, layer_idx)
        self.self_attn = OPTGQAAttention(kv_heads, heads_grouping_arr[layer_idx], config, layer_idx=layer_idx)

class CustomOPTDecoder(OPTDecoder):
    def __init__(self, config):
        super().__init__(config)
        self.layers = nn.ModuleList([CustomOPTDecoderLayer(config, layer_idx=i) for i in range(config.num_hidden_layers)])

class CustomOPTModel(OPTModel):
    def __init__(self, config):
        super().__init__(config)
        self.decoder = CustomOPTDecoder(config)

class CustomOPTForSequenceClassification(OPTForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.model = CustomOPTModel(config)
        self.post_init()


bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)

# My repo where I uploaded the GQA version of OPT.
repo_path = f"joshwapanda/{saved_path}"

model = CustomOPTForSequenceClassification.from_pretrained(repo_path, quantization_config=bnb_config, num_labels=3, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.75G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
print(model.get_memory_footprint()/1e6)
model

3384.598528


CustomOPTForSequenceClassification(
  (model): CustomOPTModel(
    (decoder): CustomOPTDecoder(
      (embed_tokens): Embedding(50272, 4096, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
      (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x CustomOPTDecoderLayer(
          (self_attn): OPTGQAAttention(
            (k_proj): Linear4bit(in_features=4096, out_features=2048, bias=True)
            (v_proj): Linear4bit(in_features=4096, out_features=2048, bias=True)
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=True)
            (out_proj): Linear4bit(in_features=4096, out_features=4096, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear4bit(in_features=4096, out_features=16384, bias=True)
          (fc2): Linear4bit(in_features=16384,

In [ ]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=16,
    bias="none",
    lora_dropout=0.05,
    task_type="SEQ_CLS",
    # manually setting target modules
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"],
)

model = get_peft_model(model, config)
print(model.get_memory_footprint()/1e6)
train_p, tot_p = model.get_nb_trainable_parameters()
print(f'Trainable parameters:      {train_p/1e6:.2f}M')
print(f'Total parameters:          {tot_p/1e6:.2f}M')
print(f'% of trainable parameters: {100*train_p/tot_p:.2f}%')
model

3845.81632
Trainable parameters:      7.35M
Total parameters:          6128.84M
% of trainable parameters: 0.12%


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): CustomOPTForSequenceClassification(
      (model): CustomOPTModel(
        (decoder): CustomOPTDecoder(
          (embed_tokens): Embedding(50272, 4096, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
          (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (layers): ModuleList(
            (0-31): 32 x CustomOPTDecoderLayer(
              (self_attn): OPTGQAAttention(
                (k_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=4096, out_features=2048, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default):

In [ ]:
tokenizer_repo_id = f"facebook/opt-{repo_size}"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_repo_id)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

# **Loading and pre-processing the dataset**

In [ ]:
from datasets import load_dataset

dataset = load_dataset("nyu-mll/glue", "mnli")
dataset

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

test_matched-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

...

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [ ]:
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(55000))

In [ ]:
def preprocess(example):
    return tokenizer(
        example["premise"],
        example["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/55000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

# **Training**

In [ ]:
model.gradient_checkpointing_enable({"use_reentrant": False})

training_args = TrainingArguments(
    output_dir="./opt-qlora-mnli",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    num_train_epochs=2,
    learning_rate=2e-4,
    warmup_steps=100,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    fp16=True,
    optim="paged_adamw_8bit",
    label_names=["labels"],
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import TrainerCallback
from sklearn.metrics import accuracy_score

class PlotLossAccuracyCallback(TrainerCallback):
    def __init__(self):
        self.train_loss = []
        self.eval_loss = []
        self.eval_acc = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return

        print(f"[LOG] Step {state.global_step}: {logs}")

        if "loss" in logs:
            self.train_loss.append((state.global_step, logs["loss"]))
        if "eval_loss" in logs:
            self.eval_loss.append((state.global_step, logs["eval_loss"]))
        if "eval_accuracy" in logs:
            self.eval_acc.append((state.global_step, logs["eval_accuracy"]))

    def plot(self):
      steps_train, loss_train = zip(*self.train_loss) if self.train_loss else ([], [])
      steps_eval, loss_eval = zip(*self.eval_loss) if self.eval_loss else ([], [])
      steps_acc, acc_eval = zip(*self.eval_acc) if self.eval_acc else ([], [])

      plt.figure(figsize=(12, 5))

      if loss_train or loss_eval:
          plt.subplot(1, 2, 1)
          if loss_train:
              plt.plot(steps_train, loss_train, label="Train Loss")
          if loss_eval:
              plt.plot(steps_eval, loss_eval, label="Eval Loss")
          plt.xlabel("Step")
          plt.ylabel("Loss")
          plt.title("Training and Eval Loss")
          plt.legend()

      if acc_eval:
          plt.subplot(1, 2, 2)
          plt.plot(steps_acc, acc_eval, label="Eval Accuracy", color="green")
          plt.xlabel("Step")
          plt.ylabel("Accuracy")
          plt.title("Validation Accuracy")
          plt.legend()

    plt.tight_layout()
    plt.savefig("loss_accuracy_plot.png")
    plt.show()

<Figure size 640x480 with 0 Axes>

In [ ]:
plot_callback = PlotLossAccuracyCallback()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation_matched"],
    tokenizer=tokenizer,
    callbacks=[plot_callback]
)

trainer.train()

/tmp/ipython-input-14-2397418827.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.497400,0.437363
2,0.385100,0.388678


[LOG] Step 100: {'loss': 1.186, 'grad_norm': 15.111870765686035, 'learning_rate': 0.000188, 'epoch': 0.23255813953488372}
[LOG] Step 200: {'loss': 0.8853, 'grad_norm': 8.891255378723145, 'learning_rate': 0.00017526315789473683, 'epoch': 0.46511627906976744}
[LOG] Step 300: {'loss': 0.5627, 'grad_norm': 6.564413547515869, 'learning_rate': 0.00014894736842105264, 'epoch': 0.6976744186046512}
[LOG] Step 400: {'loss': 0.4974, 'grad_norm': 2.4029388427734375, 'learning_rate': 0.00012263157894736842, 'epoch': 0.9302325581395349}
[LOG] Step 430: {'eval_loss': 0.43736323714256287, 'eval_runtime': 493.475, 'eval_samples_per_second': 19.89, 'eval_steps_per_second': 0.312, 'epoch': 1.0}
[LOG] Step 500: {'loss': 0.4231, 'grad_norm': 5.206781387329102, 'learning_rate': 9.631578947368421e-05, 'epoch': 1.1627906976744187}
[LOG] Step 600: {'loss': 0.3983, 'grad_norm': 4.3562846183776855, 'learning_rate': 7e-05, 'epoch': 1.3953488372093024}
[LOG] Step 700: {'loss': 0.3961, 'grad_norm': 2.81027770042419

TrainOutput(global_step=860, training_loss=0.5782070159912109, metrics={'train_runtime': 18028.3439, 'train_samples_per_second': 6.102, 'train_steps_per_second': 0.048, 'total_flos': 4.9965913079808e+17, 'train_loss': 0.5782070159912109, 'epoch': 2.0})

# **Test Model**

In [ ]:
! pip -q install evaluate
import evaluate
accuracy_metric = evaluate.load("accuracy")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation_matched"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-17-2398084693.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.evaluate()

{'eval_loss': 0.38867810368537903,
 'eval_model_preparation_time': 0.005,
 'eval_accuracy': 0.8487009679062659,
 'eval_runtime': 493.8981,
 'eval_samples_per_second': 19.873,
 'eval_steps_per_second': 0.312}